# Векторный поиск в задаче матчинга

Необходимо решить усеченную задачу матчинга - векторный поиск.

Осуществляется поиск ближайших N товаров из коллекции base для каждого товара из query.

## Подготовка данных

### Установка и импорт библиотек

In [1]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd
import faiss
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

### Константы и базовые настройки

In [3]:
CV = 4
FALSE_PART = 1

In [4]:
np.random.seed(1234)

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [6]:
os.chdir('/content/gdrive/MyDrive/Colab_Notebooks_Samokat/')

### Общая информация о датасете

Датасет содержит каталог товаров на 2.918.139 позиций.

100.000 запросов с ответами - правильными матчами (обучающая выборка).

100.000 запросов без ответов (тестовая выборка) - для проверки результатов на закрытых данных.

Каждая позиция представлена в векторном виде с 72 координатами-признаками.



In [7]:
base = pd.read_csv('base.csv')
base.shape

(2918139, 73)

In [8]:
train = pd.read_csv('train.csv')
train.shape

(100000, 74)

In [9]:
test = pd.read_csv('test.csv')
test.shape

(100000, 73)

### Отбор признаков

Благодаря наличию обучающей выборки можно выполнить анализ корреляций 72 пар координат запроса и соответствующего ему матча-товара.

In [10]:
base = base.rename(columns={'Id': 'target'})
train = train.rename(columns={'Target': 'target'})

In [11]:
df = train.merge(base, how='inner', on='target')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 146 entries, Id to 71_y
dtypes: float64(144), object(2)
memory usage: 112.2+ MB


In [13]:
incorrect_features = []

for i in range(0, 72):
  corr_result = df[f'{i}_x'].corr(df[f'{i}_y'])

  print(i, corr_result)
  if corr_result < 0.8:
    incorrect_features.append(str(i))

0 0.8449447537712971
1 0.8478439158094286
2 0.8334490344301374
3 0.8511180082446534
4 0.841223126598558
5 0.8441899324722842
6 0.6322338996868498
7 0.8377651642015352
8 0.8437878524560236
9 0.8452779026253773
10 0.8595229186576993
11 0.8337949068049615
12 0.8532650978245585
13 0.8481128821807487
14 0.8500162975996597
15 0.8521129968011875
16 0.8418913646803291
17 0.8441879416805943
18 0.8381387977799976
19 0.8452056239284101
20 0.8437645745710631
21 0.32684470188040965
22 0.8341134518158314
23 0.8418297148377708
24 0.8453072892269006
25 0.36247749147804476
26 0.8341064978470665
27 0.8395475709384276
28 0.8522746615896958
29 0.8518623720096529
30 0.8505579991706796
31 0.8535795670485254
32 0.8370928258754424
33 -0.06376557635158732
34 0.8486118071722741
35 0.8441001871901369
36 0.8467277899654437
37 0.8526558730849103
38 0.8487193551518449
39 0.8404080521291137
40 0.8396626392349058
41 0.842777972692936
42 0.8456295834098554
43 0.8475255592013271
44 0.6304830233791149
45 0.8291208939650

По результатам этого анализа видно, что по 8 из 72 координат корреляция меньше 0.8. Т.к. это координаты вектора и векторный поиск матчей сводится к поиску ближайших точек, то эти 8 координат можно отбросить как "мусорные".

В дальнейшем задача будет выполняться на 64 признаках.

In [14]:
incorrect_features

['6', '21', '25', '33', '44', '59', '65', '70']

In [15]:
del df

### Масштабирование

Масштабирую признаки для более точного поиска.

In [16]:
feature_columns = base.drop(['target'] + incorrect_features, axis=1).columns

In [17]:
scaler = StandardScaler()
scaler.fit(base[feature_columns])
base[feature_columns] = scaler.transform(base[feature_columns])
train[feature_columns] = scaler.transform(train[feature_columns])
test[feature_columns] = scaler.transform(test[feature_columns])

## Векторный поиск

Для векторного поиска использую библиотеку faiss.

In [18]:
base_dict = dict(base['target'].reset_index().set_index('target')['index'])
id_base_dict = dict(base['target'])

In [19]:
dim = 72 - len(incorrect_features)
nb = base.shape[0]
nq = train.shape[0]

Выполняется подготовка векторов.

In [20]:
vectors = base.drop(['target'] + incorrect_features, axis=1).to_numpy().astype('float32').copy(order='C')
query = train.drop(['Id', 'target'] + incorrect_features, axis=1).to_numpy().astype('float32').copy(order='C')
query_test = test.drop(['Id'] + incorrect_features, axis=1).to_numpy().astype('float32').copy(order='C')

Т.к. будет использоваться косинусное расстояние, векторы нормализуются (это необходимо согласно документации).

In [21]:
faiss.normalize_L2(vectors)
faiss.normalize_L2(query)
faiss.normalize_L2(query_test)

### GPU

Индекс строю на GPU - ресурсов Colab для этого более чем достаточно, поиск выполняется быстро, а полный перебор по понятным причинам дает лучшие метрики.

In [22]:
res = faiss.StandardGpuResources()

In [23]:
index = faiss.IndexFlatIP(dim)
gpu_index = faiss.index_cpu_to_gpu(res, 0, index)
gpu_index.add(vectors)

Для каждого запроса получаю топ-10 матчей.

In [24]:
topn = 10

In [25]:
%%time
D, I = gpu_index.search(query, topn) 

CPU times: user 13.4 s, sys: 6.01 s, total: 19.4 s
Wall time: 20.1 s


По итогам поиска считаю метрику на обучающей выборке:

Recall@10 = (общая сумма найденных в топ10 матчей) / (общая сумма матчей)

In [26]:
results = pd.DataFrame()
results['target'] = train['target']
results['top10'] = list(I)
results['true'] = results.apply(lambda x: 1 if base_dict[x['target']] in x['top10'] else 0, axis=1)
results['position'] = results.apply(lambda x: list(x['top10']).index(base_dict[x['target']]) if base_dict[x['target']] in x['top10'] else -1, axis=1)

In [27]:
recall = results['true'].sum()/results.shape[0]
recall

0.73977

Видно, что большая часть правильных матчей приходится на 1-3 позицию в топ-10.

In [28]:
results['position'].value_counts().head(50)

 0    59578
-1    26023
 1     6196
 2     2684
 3     1615
 4     1081
 5      804
 6      651
 7      541
 8      438
 9      389
Name: position, dtype: int64

In [29]:
del results

## Получение результатов по тестовой выборке для отправки

In [30]:
%%time
D, I = gpu_index.search(query_test, topn) 

CPU times: user 14.3 s, sys: 6.55 s, total: 20.8 s
Wall time: 20.9 s


In [31]:
#из таблицы I с индексами кандидатов формируем строку в формате ответа
                    
predicted_list = []
for candidates in I:
    predicted_list.append(' '.join([id_base_dict[candidate] for candidate in candidates]))
    
#формируем ответ
baseline_answer = test[['Id']]
baseline_answer['Predicted'] = predicted_list

<ipython-input-31-1d26d5f93ac6>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  baseline_answer['Predicted'] = predicted_list


In [32]:
baseline_answer.to_csv('answer_test.csv', index=False)

## Ранжирование

Для получения более точных результатов можно получить, например, топ-100 матчей вместо топ-10 и проранжировать их с помощью обученной модели.

Моя идея была в следующем:
* на основе обучающей выборки собрать 64 признака товара и 64 признака запроса - это матчи с таргетом 1.
* для таргета 0 - собрать 128 аналогичных признаков для случайных пар из base (запрос - из первой половины base, матч - из второй половины base).
* на полученных данных обучить модель логистической регрессии.
* с помощью модели получить вероятности для каждой пары запрос - кандидат из топ-100.
* отранжировать кандидатов по вероятностям и выбрать из них топ-10.

Получаю первую часть данных для обучения модели - таргет 1 на основе исходной обучающей выборки.

In [33]:
df_scaller = train.merge(base, how='inner', on='target')
df_scaller.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 146 entries, Id to 71_y
dtypes: float64(144), object(2)
memory usage: 112.2+ MB


In [34]:
feature_columns

Index(['0', '1', '2', '3', '4', '5', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '22', '23', '24', '26', '27',
       '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '40',
       '41', '42', '43', '45', '46', '47', '48', '49', '50', '51', '52', '53',
       '54', '55', '56', '57', '58', '60', '61', '62', '63', '64', '66', '67',
       '68', '69', '71'],
      dtype='object')

In [35]:
lr_features = [f'{x}_x' for x in feature_columns] + [f'{x}_y' for x in feature_columns]

In [36]:
train_true = df_scaller[lr_features]#.sample(n=20000, random_state=12345).reset_index(drop=True)

In [37]:
del df_scaller

In [38]:
train_true['target'] = 1

<ipython-input-38-74bcc60548c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_true['target'] = 1


In [39]:
train_true.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 129 entries, 0_x to target
dtypes: float64(128), int64(1)
memory usage: 99.2 MB


Перед получением второй части с таргетом 0 удаляю из каталога товаров base дубли (они появились после удаления 8 "мусорных" координат).

In [40]:
base_wo_duples = base[feature_columns].drop_duplicates()

In [41]:
base_wo_duples.shape

(2539424, 64)

Определяю размер части с таргетом 0 относительно размера части с таргетом 1 (с помощью регулирования константы FALSE_PART можно менять баланс классов).

In [42]:
sample_size = FALSE_PART * train_true.shape[0]

Из первой половины каталога получаю запросы, из второй - матчи. У таких пар таргет равен 0. Итог - вторая часть данных для обучения модели.

In [43]:
sample_query = base_wo_duples[0:1250001].sample(n=sample_size, random_state=12345).reset_index(drop=True)

In [44]:
sample_query.shape

(100000, 64)

In [45]:
sample_base = base_wo_duples[1250001:base_wo_duples.shape[0]].sample(n=sample_size, random_state=12345).reset_index(drop=True)

In [46]:
sample_base.shape

(100000, 64)

In [47]:
del base_wo_duples

In [48]:
train_false = sample_query.join(sample_base, lsuffix='_x', rsuffix='_y')

In [49]:
del sample_query, sample_base

In [50]:
train_false['target'] = 0

Объединяю части и получаю выборку для обучения логистической регрессии.

In [51]:
train_for_range = shuffle(pd.concat([train_true, train_false]), random_state=12345).reset_index(drop=True)

In [52]:
train_for_range.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 129 entries, 0_x to target
dtypes: float64(128), int64(1)
memory usage: 196.8 MB


In [53]:
del train_true, train_false

In [54]:
features = train_for_range[lr_features]
target = train_for_range['target']

In [55]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=12345)

In [56]:
del train_for_range
del features
del target

Обучаю логистическу регрессию.

In [57]:
model = LogisticRegression(random_state=12345, solver='lbfgs')#, class_weight = 'balanced')
scores = cross_val_score(model, features_train, target_train, scoring='recall', cv=CV)
round(pd.Series(scores).mean(), 3)

0.587

In [58]:
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print(recall_score(target_test, predictions))

0.5868446043884641


In [59]:
del features_train, features_test, target_train, target_test

### Качество ранжирования на исходной обучающей выборке с топ-15

In [60]:
topn = 15

In [61]:
%%time
D, I = gpu_index.search(query, topn) 

CPU times: user 14.8 s, sys: 6.79 s, total: 21.6 s
Wall time: 21.7 s


По итогам поиска считаю метрику на обучающей выборке - для топ-15:

Recall@15 = (общая сумма найденных в топ15 матчей) / (общая сумма матчей)

Для топ-10 было значение 0.73977

In [62]:
results = pd.DataFrame()
results['target'] = train['target']
results['top10'] = list(I)
results['true'] = results.apply(lambda x: 1 if base_dict[x['target']] in x['top10'] else 0, axis=1)
results['position'] = results.apply(lambda x: list(x['top10']).index(base_dict[x['target']]) if base_dict[x['target']] in x['top10'] else -1, axis=1)

In [63]:
recall = results['true'].sum()/results.shape[0]
recall

0.75413

In [64]:
del results

Готовлю признаки для предсказания моделью.

In [65]:
df_candidates = train[['Id'] + list(feature_columns)]

In [66]:
df_candidates['target_index'] = list(I)

<ipython-input-66-6ac1ace7dbb4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_candidates['target_index'] = list(I)


In [67]:
df_candidates = df_candidates.explode("target_index", ignore_index=True)

In [68]:
df_candidates['target'] = df_candidates['target_index'].apply(lambda x: id_base_dict[x])

In [69]:
df_candidates.head()

,Id,0,1,2,3,4,5,7,8,9,...,62,63,64,66,67,68,69,71,target_index,target
0,0-query,1.299519,1.996888,0.063774,-1.879671,1.6441,-0.537626,0.279598,-2.296794,-1.459626,...,-0.866975,1.274319,-0.02441,-1.035388,0.197184,-0.200786,0.906575,0.522963,336969,361564-base
1,0-query,1.299519,1.996888,0.063774,-1.879671,1.6441,-0.537626,0.279598,-2.296794,-1.459626,...,-0.866975,1.274319,-0.02441,-1.035388,0.197184,-0.200786,0.906575,0.522963,1113711,1375561-base
2,0-query,1.299519,1.996888,0.063774,-1.879671,1.6441,-0.537626,0.279598,-2.296794,-1.459626,...,-0.866975,1.274319,-0.02441,-1.035388,0.197184,-0.200786,0.906575,0.522963,1818641,2515747-base
3,0-query,1.299519,1.996888,0.063774,-1.879671,1.6441,-0.537626,0.279598,-2.296794,-1.459626,...,-0.866975,1.274319,-0.02441,-1.035388,0.197184,-0.200786,0.906575,0.522963,480296,530165-base
4,0-query,1.299519,1.996888,0.063774,-1.879671,1.6441,-0.537626,0.279598,-2.296794,-1.459626,...,-0.866975,1.274319,-0.02441,-1.035388,0.197184,-0.200786,0.906575,0.522963,232405,244376-base


In [70]:
candidates_features = df_candidates.merge(base, on='target')[lr_features] 

In [71]:
df_candidates = df_candidates[['Id', 'target']]

Получаю вероятности.

In [72]:
probabilities_valid = model.predict_proba(candidates_features)
probabilities_one_valid = probabilities_valid[:, 1]
probabilities_one_valid

array([0.60554368, 0.60792227, 0.59095321, ..., 0.48954183, 0.51774251,
       0.48435692])

In [73]:
del candidates_features

In [74]:
df_candidates['probability'] = probabilities_one_valid

Сортирую по убыванию вероятностей и получаю отранжированный топ-10.

In [75]:
df_candidates = df_candidates.sort_values(by=['Id', 'probability'], ascending=[True, False])

In [76]:
df_candidates['number'] = df_candidates.groupby('Id').cumcount()

In [77]:
results = df_candidates[df_candidates['number'] < 10]

In [90]:
results.head(20)

,Id,target,probability,number
3,0-query,530165-base,0.631472,0
11,0-query,91611-base,0.613592,1
4,0-query,244376-base,0.612901,2
1,0-query,1375561-base,0.607922,3
6,0-query,3543241-base,0.606616,4
0,0-query,361564-base,0.605544,5
12,0-query,1668267-base,0.603633,6
9,0-query,877519-base,0.602835,7
10,0-query,715085-base,0.598321,8
14,0-query,1265224-base,0.595532,9


In [79]:
results1 = results.copy()

In [80]:
results1['top10'] = results1.groupby('Id').transform(lambda x: ' '.join(x))
results1 = results1[['Id', 'top10']].drop_duplicates().reset_index(drop=True)
results1.head()

<ipython-input-80-6c3be2a95eba>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  results1['top10'] = results1.groupby('Id').transform(lambda x: ' '.join(x))


,Id,top10
0,0-query,530165-base 91611-base 244376-base 1375561-bas...
1,1-query,15226-base 165320-base 356353-base 854272-base...
2,10-query,3554046-base 3006692-base 1790410-base 418450-...
3,100-query,1899-base 2423917-base 1811437-base 1211397-ba...
4,1000-query,179355-base 3342261-base 3727283-base 1653160-...


В итоге метрика на топ-10 после ранжирования сильно хуже чем при получении топ-10 только векторным поиском.

In [85]:
results1 = results1.merge(train[['Id', 'target']], how='inner', on='Id')

In [86]:
results1['true'] = results1.apply(lambda x: 1 
                                if x['target'] in x['top10'].split(' ') else 0, axis=1)

In [88]:
recall = results1['true'].sum()/results1.shape[0]
recall

0.49648

К сожалению, мне не хватило памяти, чтобы брать большее количество кандидатов для ранжирования, чем топ-15, и чтобы поэкспериментировать с разным объемом и балансом классов для логистической регрессии.

А для оптимизации (чтоб хватало памяти) не хватило уже времени (:(

Большое спасибо за уделенное время и обратную связь.

И мне очень понравилась задача и организация воркшопа. Приятно было сотрудничать с командой Самоката и Практикума.

